In [11]:
import carla
import math
import random
import time
import numpy as np
import open3d as o3d

In [12]:
#Conexion cliente-servidor
client = carla.Client('localhost', 2000) #cliente
world = client.get_world() #world reprsenta la simulacion, server

settings = world.get_settings()
print(settings)

WorldSettings(synchronous_mode=False,no_rendering_mode=False,fixed_delta_seconds=4.4749e-318,substepping=True,max_substep_delta_time=0.01,max_substeps=10,max_culling_distance=0,deterministic_ragdolls=True)


In [13]:
#settings de simulacion

#fixed delta - tiempo entre pasos de simulacion fijo
#modo sincrono - el server espera un tick del cliente para ejecutar un paso de simulacion

delta = 0.05
settings.fixed_delta_seconds = delta
settings.synchronous_mode = True

world.apply_settings(settings)

20

In [14]:
#Libreria de blueprints, para spawnear vehiculos, sensores, etc
bp_lib = world.get_blueprint_library()

In [15]:
#sensor lidar
#Velodyne HDL-64
lidar_bp = bp_lib.find('sensor.lidar.ray_cast')
#lidar_bp.set_attribute('sensor_tick', '1.0') # 1 segundo tic
lidar_bp.set_attribute('channels',str(64))
lidar_bp.set_attribute('upper_fov',str(2.0))
lidar_bp.set_attribute('lower_fov',str(-24.9))
lidar_bp.set_attribute('points_per_second',str(1300000)) #Single Return Mode
lidar_bp.set_attribute('rotation_frequency',str(1.0/delta)) #frec de sim = 1/fixed_delta_time = 10 debe coincidir con la frecuencia de rotación para generar una vuelta completa por paso de simulacion
lidar_bp.set_attribute('range',str(120))
lidar_bp.set_attribute('dropoff_general_rate','0.0')
lidar_bp.set_attribute('noise_stddev', '0.01')

In [16]:
#se puede obtener la ubicacion de la camara durante la simulacion
spectator = world.get_spectator()
spectator_transform = spectator.get_transform()
print(spectator_transform)

Transform(Location(x=0.000000, y=0.000000, z=1.120007), Rotation(pitch=0.000000, yaw=0.000000, roll=0.000000))


In [17]:
#Spawnear el sensor, ubicacion esqquina de pruebas en TOWN2
transform = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.5), 
                            carla.Rotation(pitch=0.0, yaw=0.0, roll=0.0))
sensor_raycast = world.spawn_actor(lidar_bp, transform)

In [18]:
def lidar_callback(point_cloud):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    print(point_cloud)
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]
    print(intensity)
    colors = []
    for i in range(len(intensity)):
        red=intensity[i]
        blue=1.0-intensity[i]
        colors.append((red,0.0,blue))
        
        #if (intensity[i]<0.5):
            #colors.append((0.0,0.0,1.0))
        #else:
            #colors.append((1.0,0.0,0.0))

    # Isolate the 3D data
    points = data[:, :-1]
    # We're negating the y to correclty visualize a world that matches
    # what we see in Unreal since Open3D uses a right-handed coordinate system
    points[:, :1] = -points[:, :1]
    
    #point cloud para visualizar
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    o3d.visualization.draw_geometries([pcd])

In [19]:

sensor_raycast.listen(lambda data: lidar_callback(data))

In [22]:
# # This ca fix Open3D jittering issues:
time.sleep(0.005)
world.tick()


LidarMeasurement(frame=24, timestamp=21.958736, number_of_points=32287)
[1.  1.  1.  ... 0.2 0.2 0.2]


24

TypeError: lidar_callback() takes 1 positional argument but 2 were given

In [21]:
colores=[]
intensidad = 1.0
red = intensidad
blue = 1.0 - intensidad
colores.append((red,0.0,blue))

print(colores)

[(1.0, 0.0, 0.0)]


TypeError: lidar_callback() takes 1 positional argument but 2 were given